In [11]:
# This script take 4 arguments:
# python parser.py logs1  logs2  logs3 output.csv
# This join the 3 logs and parse the queries, the output.csv contains non duplicated queries csv ['id','query','time'=1]

import pandas as pd
import numpy as np
from urllib.parse import urlparse,unquote
import sys
import re

# "./example_logs_organic.tsv"
dataset1 = pd.read_csv(sys.argv[1],sep="\t")
dataset2 = pd.read_csv(sys.argv[2],sep="\t")
dataset3 = pd.read_csv(sys.argv[3],sep="\t")
dataset = pd.concat([dataset1,dataset2,dataset3], ignore_index=True)

def clean_query(x):
    parsed = unquote(x)
    parsed = re.sub('\+', ' ', parsed)
    parsed = re.sub('\n+', ' ', parsed)
    parsed = re.sub(' +', ' ', parsed)
    return parsed.strip()
dataset['query'] = dataset['anonymizedQuery'].apply(lambda x: clean_query(x))
print(dataset.shape)
dataset = dataset[['query']].drop_duplicates('query')
print(dataset.shape)
dataset = dataset[~dataset['query'].str.contains("MINUS")]
print(dataset.shape)
dataset = dataset[~dataset['query'].str.contains("SERVICE")]
print(dataset.shape)

dataset = dataset[~dataset['query'].str.contains("BIND")]
print(dataset.shape)
import hashlib
import csv
def hash_query(x):
    # Assumes the default UTF-8
    hash_object = hashlib.md5(x.encode())
    return hash_object.hexdigest()
dataset['id'] = dataset['query'].apply(lambda x: hash_query(x))
dataset['time'] = dataset['query'].apply(lambda x: 1)
dataset[['id','query','time']].to_csv(sys.argv[4], index=None,sep="ᶶ", quoting=3)